<a href="https://colab.research.google.com/github/amaanhash/Profanity-Filter-for-kids-using-video-sharing-platform/blob/main/FaceDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from tensorflow.keras.utlis import load_img
from keras.model import Sequential,Model
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import seaborn as sns
import warnings
from tqdm.noteobok import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline


ModuleNotFoundError: No module named 'tensorflow.keras.utlis'

In [ ]:
from google.colab import drive